In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [2]:
def make_num_list(df):
    num_list = []
    i = 1
    while i < len(df)+1:
        num_list.append(i)
        i = i + 1
    df_new = df.copy()
    df_new['num'] = num_list
    return df_new

def time_list(df, t1, t2, graph): # 1-обычный, 2 - триграф
    df = make_num_list(df)
    t_list = []
    i = 1
    while i < len(df) - graph + 1:
        new_time = (int(df.loc[df['num'] == (i+graph)][t1])-int(df.loc[df['num'] == i][t2]))/1000
        t_list.append(new_time)
        i = i + 1
    while graph > 0:
        t_list.append(0)
        graph = graph - 1
    return t_list

def create_and_save_key(df):
    df_key = df.loc[df['action_type'] == 'key']
    df_key_time = df_key[['user_id', 'session', 'action_type', 'key_code', 'down', 'up', 'time']]
#    df_key_time.to_csv('key.csv')
    return df_key_time

def do_features(df):
    df['DD'] = time_list(df, 'down', 'down', 1)
    df['UD'] = time_list(df, 'down', 'up', 1)
    df['UU'] = time_list(df, 'up', 'up', 1)
    df['DU'] = time_list(df, 'up', 'down', 1)
    df['DD3'] = time_list(df, 'down', 'down', 2)
    df['UD3'] = time_list(df, 'down', 'up', 2)
    df['UU3'] = time_list(df, 'up', 'up', 2)
    df['DU3'] = time_list(df, 'up', 'down', 2)
    df['DD4'] = time_list(df, 'down', 'down', 3)
    df['UD4'] = time_list(df, 'down', 'up', 3)
    df['UU4'] = time_list(df, 'up', 'up', 3)
    df['DU4'] = time_list(df, 'up', 'down', 3)
    df['DD5'] = time_list(df, 'down', 'down', 4)
    df['UD5'] = time_list(df, 'down', 'up', 4)
    df['UU5'] = time_list(df, 'up', 'up', 4)
    df['DU5'] = time_list(df, 'up', 'down', 4)
    df['kpm5'] = kpm(df, 5)
    df['kpm10'] = kpm(df, 10)
    df['skew5_time'] = skew_text_input(df, 5, 'time')
    df['skew10_time'] = skew_text_input(df, 10, 'time')
    df['skew20_time'] = skew_text_input(df, 10, 'time')
    df['skew5_kpm5'] = skew_text_input(df, 5, 'kpm5')
    df['skew10_kpm5'] = skew_text_input(df, 10, 'kpm5')
    df['skew5_kpm10'] = skew_text_input(df, 5, 'kpm10')
    df['skew10_kpm10'] = skew_text_input(df, 10, 'kpm10')
    df['error_rate'] = error_rate(df, 20)
    return df

def kpm(df, num):
    df = make_num_list(df)
    v = []
    i = 1
    while i < len(df) - num + 1:
        t = (int(df.loc[df['num'] == (i+num-1)]['up'])-int(df.loc[df['num'] == i]['down']))/1000
        new_v = num/t
        v.append(new_v*60)
        i = i + 1
    while num > 0:
        t = (int(df.loc[df['num'] == (i+num-2)]['up'])-int(df.loc[df['num'] == i]['down']))/1000
        new_v = num/t
        v.append(new_v)
        i += 1
        num = num - 1
    return v

def skew_text_input(df, num, feature):
    s = []
    list_feature = list(df[feature])
    i = 0
    while i < len(df) - num + 1:
        s.append(skew(list_feature[i:i+num]))
        i = i + 1
    while num > 1:        
        s.append(skew(list_feature[i:i+num-1]))
        i += 1
        num = num - 1
    return s

def num_of_error(l):
    num = 0
    for i in l:
        if 0<i<3:
            num += 1
    return num

def error_rate(df, num):
    error_rate_list = []
    key_code = list(df['key_code'])
    i = 0
    while i < len(df) - num + 1:
        rate = num_of_error(key_code[i:i+num])/num
        error_rate_list.append(rate)
        i = i + 1
    while num > 1:
        error_rate_list.append(num_of_error(key_code[i:i+num-1])/(num-1))
        i += 1
        num = num - 1
    return error_rate_list

def create_and_save_mouse(df, name):
    df_mouse = df.loc[df['action_type'] == 'mouse']
    df_mouse_time = df_mouse[['user_id', 'session', 'action_type', 'key_code', 'down', 'up', 'time']]
    df_mouse_time.to_csv(name + '.csv')
    return df_mouse_time

def remove_outlier (df):
    features = ['time', 'UD', 'DD3', 'DD4', 'DD5']
    for f in features:
        bottom = np.percentile(df[f], 5)
        top = np.percentile(df[f], 95)
        df = df.loc[(bottom < df[f])  & (df[f] < top)].reset_index(drop=True)
    return df

def print_box (df):
    col_name = list(df.columns[7:])
    for f in col_name:
        plt.figure()
        df[['session', f]].boxplot(column=[f], figsize=(6,8))

In [3]:
df1 = pd.read_csv('datasets\dataset_89270693456.csv', delimiter=',') #Денис_л
df2 = pd.read_csv('datasets\dataset_89196230374.csv', delimiter=',') #Никита_л
df3 = pd.read_csv('datasets\dataset_89270698741.csv', delimiter=',') #Максим_л
df4 = pd.read_csv('datasets\dataset_80171109282.csv', delimiter=',') #Максим_у
df5 = pd.read_csv('datasets\dataset_89176732344.csv', delimiter=',') #Даша_л
df6 = pd.read_csv('datasets\dataset_89273345194.csv', delimiter=',') #Даша_у
df7 = pd.read_csv('datasets\dataset_89162735523.csv', delimiter=',') #Сергей_л
df8 = pd.read_csv('datasets\dataset_89193729288.csv', delimiter=',') #Алина_л
df9 = pd.read_csv('datasets\dataset_89273028630.csv', delimiter=',') #Люба_л
df10 = pd.read_csv('datasets\dataset_89163828720.csv', delimiter=',') #Люба_у

In [4]:
df1['type'] = 0
df2['type'] = 0
df3['type'] = 0
df4['type'] = 1
df5['type'] = 0
df6['type'] = 1
df7['type'] = 0
df8['type'] = 0
df9['type'] = 0
df10['type'] = 1

In [5]:
dataframe = df1
dataframe = dataframe.merge(df2, how='outer')
dataframe = dataframe.merge(df3, how='outer')
dataframe = dataframe.merge(df4, how='outer')
dataframe = dataframe.merge(df5, how='outer')
dataframe = dataframe.merge(df6, how='outer')
dataframe = dataframe.merge(df7, how='outer')
dataframe = dataframe.merge(df8, how='outer')
dataframe = dataframe.merge(df9, how='outer')
dataframe = dataframe.merge(df10, how='outer')

In [6]:
dataframe.describe()

,user_id,session,key_code,down,up,time,x,y,type
count,3.522200e+04,35222.000000,2694.000000,8.892000e+03,8.892000e+03,3.522200e+04,26330.000000,26330.000000,35222.000000
mean,8.867756e+10,6.239424,1.264291,1.606846e+12,1.606846e+12,1.201122e+12,698.258982,422.317888,0.235506
std,2.153812e+09,3.974972,1.756831,4.853294e+08,4.853293e+08,6.980198e+11,260.797940,126.825179,0.424321
min,8.017111e+10,1.000000,1.000000,1.606151e+12,1.606151e+12,0.000000e+00,0.000000,0.000000,0.000000
25%,8.916383e+10,3.000000,1.000000,1.606558e+12,1.606558e+12,6.470000e-01,474.000000,337.000000,0.000000
50%,8.919373e+10,6.000000,1.000000,1.606586e+12,1.606586e+12,1.606560e+12,672.000000,426.000000,0.000000
75%,8.927070e+10,9.000000,1.000000,1.607266e+12,1.607266e+12,1.607264e+12,935.000000,512.000000,0.000000
max,8.927335e+10,18.000000,13.000000,1.607589e+12,1.607589e+12,1.607589e+12,1918.000000,731.000000,1.000000


In [7]:
user_id = dataframe['user_id'].unique()
action_type = dataframe['action_type'].unique()

In [8]:
for i in user_id:
    print('Количество событий key ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'key') & (dataframe['user_id'] == i)]))
print('Среднее число событй key на одного пользователя: ' + str(len(dataframe.loc[dataframe['action_type'] == 'key'])/len(user_id)))
print('Общее число событй key: ' + str(len(dataframe.loc[dataframe['action_type'] == 'key'])))

Количество событий key 89270693456
743
Количество событий key 89196230374
247
Количество событий key 89270698741
619
Количество событий key 80171109282
661
Количество событий key 89176732344
970
Количество событий key 89273345194
1355
Количество событий key 89162735523
1246
Количество событий key 89193729288
803
Количество событий key 89273028630
565
Количество событий key 89163828720
630
Среднее число событй key на одного пользователя: 783.9
Общее число событй key: 7839


In [35]:
for i in user_id:
    print('Количество локальных событий key ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'key') & (dataframe['user_id'] == i) & (dataframe['type'] == 0)]))
print('Среднее число событй key при локальном покдлючении на одного пользователя: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'key') & (dataframe['type'] == 0)])/len(user_id)))
print('Общее число локальных событй key: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'key') & (dataframe['type'] == 0)])))

Количество локальных событий key 89270693456
743
Количество локальных событий key 89196230374
247
Количество локальных событий key 89270698741
619
Количество локальных событий key 80171109282
0
Количество локальных событий key 89176732344
970
Количество локальных событий key 89273345194
0
Количество локальных событий key 89162735523
1246
Количество локальных событий key 89193729288
803
Количество локальных событий key 89273028630
565
Количество локальных событий key 89163828720
0
Среднее число событй key при локальном покдлючении на одного пользователя: 519.3
Общее число локальных событй key: 5193


In [36]:
for i in user_id:
    print('Количество удалённых событий key ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'key') & (dataframe['user_id'] == i) & (dataframe['type'] == 1)]))
print('Среднее число событй key при удалённом покдлючении на одного пользователя: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'key') & (dataframe['type'] == 1)])/len(user_id)))
print('Общее число удалённых событй key: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'key') & (dataframe['type'] == 1)])))

Количество удалённых событий key 89270693456
0
Количество удалённых событий key 89196230374
0
Количество удалённых событий key 89270698741
0
Количество удалённых событий key 80171109282
661
Количество удалённых событий key 89176732344
0
Количество удалённых событий key 89273345194
1355
Количество удалённых событий key 89162735523
0
Количество удалённых событий key 89193729288
0
Количество удалённых событий key 89273028630
0
Количество удалённых событий key 89163828720
630
Среднее число событй key при удалённом покдлючении на одного пользователя: 264.6
Общее число удалённых событй key: 2646


In [39]:
for i in user_id:
    print('Количество событий mouse для ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'mouse') & (dataframe['user_id'] == i)]))
print('Среднее число событй mouse на одного пользователя: ' + str(len(dataframe.loc[dataframe['action_type'] == 'mouse'])/len(user_id)))
print('Общее число событй mouse: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'mouse')])))

Количество событий mouse для 89270693456
93
Количество событий mouse для 89196230374
33
Количество событий mouse для 89270698741
124
Количество событий mouse для 80171109282
104
Количество событий mouse для 89176732344
119
Количество событий mouse для 89273345194
163
Количество событий mouse для 89162735523
156
Количество событий mouse для 89193729288
105
Количество событий mouse для 89273028630
71
Количество событий mouse для 89163828720
85
Среднее число событй mouse на одного пользователя: 105.3
Общее число событй mouse: 1053


In [41]:
for i in user_id:
    print('Количество локальных событий mouse для ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'mouse') & (dataframe['user_id'] == i) & (dataframe['type'] == 0)]))
print('Среднее число локальных событий mouse на одного пользователя: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'mouse') & (dataframe['type'] == 0)])/len(user_id)))
print('Общее число локальных событй mouse: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'mouse') & (dataframe['type'] == 0)])))

Количество локальных событий mouse для 89270693456
93
Количество локальных событий mouse для 89196230374
33
Количество локальных событий mouse для 89270698741
124
Количество локальных событий mouse для 80171109282
0
Количество локальных событий mouse для 89176732344
119
Количество локальных событий mouse для 89273345194
0
Количество локальных событий mouse для 89162735523
156
Количество локальных событий mouse для 89193729288
105
Количество локальных событий mouse для 89273028630
71
Количество локальных событий mouse для 89163828720
0
Среднее число локальных событий mouse на одного пользователя: 70.1
Общее число локальных событй mouse: 701


In [45]:
for i in user_id:
    print('Количество удалённых событий mouse для ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'mouse') & (dataframe['user_id'] == i) & (dataframe['type'] == 1)]))
print('Среднее число удалённых событий mouse на одного пользователя: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'mouse') & (dataframe['type'] == 1)])/len(user_id)))
print('Общее число удалённых событий mouse: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'mouse') & (dataframe['type'] == 1)])))

Количество удалённых событий mouse для 89270693456
0
Количество удалённых событий mouse для 89196230374
0
Количество удалённых событий mouse для 89270698741
0
Количество удалённых событий mouse для 80171109282
104
Количество удалённых событий mouse для 89176732344
0
Количество удалённых событий mouse для 89273345194
163
Количество удалённых событий mouse для 89162735523
0
Количество удалённых событий mouse для 89193729288
0
Количество удалённых событий mouse для 89273028630
0
Количество удалённых событий mouse для 89163828720
85
Среднее число удалённых событий mouse на одного пользователя: 35.2
Общее число удалённых событий mouse: 352


In [47]:
for i in user_id:
    print('Количество событий cursor для ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'cursor') & (dataframe['user_id'] == i)]))
print('Среднее число событй cursor на одного пользователя: ' + str(len(dataframe.loc[dataframe['action_type'] == 'cursor'])/len(user_id)))
print('Общее число событий cursor: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'cursor')])))

Количество событий cursor для 89270693456
3047
Количество событий cursor для 89196230374
1192
Количество событий cursor для 89270698741
3425
Количество событий cursor для 80171109282
1356
Количество событий cursor для 89176732344
3022
Количество событий cursor для 89273345194
3386
Количество событий cursor для 89162735523
4563
Количество событий cursor для 89193729288
3452
Количество событий cursor для 89273028630
2332
Количество событий cursor для 89163828720
555
Среднее число событй cursor на одного пользователя: 2633.0
Общее число событий cursor: 26330


In [48]:
for i in user_id:
    print('Количество локальных событий cursor для ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'cursor') & (dataframe['user_id'] == i) & (dataframe['type'] == 0)]))
print('Среднее число локальных событий cursor на одного пользователя: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'cursor') & (dataframe['type'] == 0)])/len(user_id)))
print('Общее число локальных событий cursor: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'cursor') & (dataframe['type'] == 0)])))

Количество локальных событий cursor для 89270693456
3047
Количество локальных событий cursor для 89196230374
1192
Количество локальных событий cursor для 89270698741
3425
Количество локальных событий cursor для 80171109282
0
Количество локальных событий cursor для 89176732344
3022
Количество локальных событий cursor для 89273345194
0
Количество локальных событий cursor для 89162735523
4563
Количество локальных событий cursor для 89193729288
3452
Количество локальных событий cursor для 89273028630
2332
Количество локальных событий cursor для 89163828720
0
Среднее число локальных событй cursor на одного пользователя: 2103.3
Общее число локальных событий cursor: 21033


In [49]:
for i in user_id:
    print('Количество удалённых событий cursor для ' + str(i))
    print(len(dataframe.loc[(dataframe['action_type'] == 'cursor') & (dataframe['user_id'] == i) & (dataframe['type'] == 1)]))
print('Среднее число удалённых событий cursor на одного пользователя: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'cursor') & (dataframe['type'] == 1)])/len(user_id)))
print('Общее число удалённых событий cursor: ' + str(len(dataframe.loc[(dataframe['action_type'] == 'cursor') & (dataframe['type'] == 1)])))

Количество удалённых событий cursor для 89270693456
0
Количество удалённых событий cursor для 89196230374
0
Количество удалённых событий cursor для 89270698741
0
Количество удалённых событий cursor для 80171109282
1356
Количество удалённых событий cursor для 89176732344
0
Количество удалённых событий cursor для 89273345194
3386
Количество удалённых событий cursor для 89162735523
0
Количество удалённых событий cursor для 89193729288
0
Количество удалённых событий cursor для 89273028630
0
Количество удалённых событий cursor для 89163828720
555
Среднее число удалённых событий cursor на одного пользователя: 529.7
Общее число удалённых событий cursor: 5297


In [11]:
dataframe.describe()

,user_id,session,key_code,down,up,time,x,y,type
count,3.522200e+04,35222.000000,2694.000000,8.892000e+03,8.892000e+03,3.522200e+04,26330.000000,26330.000000,35222.000000
mean,8.867756e+10,6.239424,1.264291,1.606846e+12,1.606846e+12,1.201122e+12,698.258982,422.317888,0.235506
std,2.153812e+09,3.974972,1.756831,4.853294e+08,4.853293e+08,6.980198e+11,260.797940,126.825179,0.424321
min,8.017111e+10,1.000000,1.000000,1.606151e+12,1.606151e+12,0.000000e+00,0.000000,0.000000,0.000000
25%,8.916383e+10,3.000000,1.000000,1.606558e+12,1.606558e+12,6.470000e-01,474.000000,337.000000,0.000000
50%,8.919373e+10,6.000000,1.000000,1.606586e+12,1.606586e+12,1.606560e+12,672.000000,426.000000,0.000000
75%,8.927070e+10,9.000000,1.000000,1.607266e+12,1.607266e+12,1.607264e+12,935.000000,512.000000,0.000000
max,8.927335e+10,18.000000,13.000000,1.607589e+12,1.607589e+12,1.607589e+12,1918.000000,731.000000,1.000000


In [12]:
print('Колчиество событий при локальном подключении: ' + str(len(dataframe.loc[dataframe['type'] == 0])))
print('Колчиество событий при удалённом подключении: ' + str(len(dataframe.loc[dataframe['type'] == 1])))

Колчиество событий при локальном подключении: 26927
Колчиество событий при удалённом подключении: 8295


In [13]:
print('Колчиество событий при локальном подключении на одного пользователя: ' + str(len(dataframe.loc[dataframe['type'] == 0])/len(user_id)))
print('Колчиество событий при удалённом подключении на одного пользователя: ' + str(len(dataframe.loc[dataframe['type'] == 1])/len(user_id)))
print('Колчиество событий на одного пользователя: ' + str(len(dataframe)/len(user_id)))

Колчиество событий при локальном подключении на одного пользователя: 2692.7
Колчиество событий при удалённом подключении на одного пользователя: 829.5
Колчиество событий на одного пользователя: 3522.2


In [14]:
s_num = 0
for i in user_id:
    print('Количество сессий для ' + str(i))
    print(list(dataframe['session'].loc[(dataframe['user_id'] == i)])[-1])
    s_num += int(list(dataframe['session'].loc[(dataframe['user_id'] == i)])[-1])
print('Число сессий: ' + str(s_num))
print('Среднее число сессий на одного пользователя: ' + str(s_num/len(user_id)))


Количество сессий для 89270693456
10
Количество сессий для 89196230374
4
Количество сессий для 89270698741
10
Количество сессий для 80171109282
10
Количество сессий для 89176732344
13
Количество сессий для 89273345194
18
Количество сессий для 89162735523
14
Количество сессий для 89193729288
9
Количество сессий для 89273028630
9
Количество сессий для 89163828720
9
Число сессий: 106
Среднее число сессий на одного пользователя: 10.6


In [20]:
s_loc = 0
data_0 = dataframe.loc[dataframe['type'] == 0]
user_loc = data_0['user_id'].unique()
for i in user_loc:
    print('Количество локальных сессий для ' + str(i))
    print(list(data_0['session'].loc[(data_0['user_id'] == i)])[-1])
    s_loc += int(list(data_0['session'].loc[(data_0['user_id'] == i)])[-1])
print('Число сессий: ' + str(s_loc))
print('Среднее число сессий на одного пользователя: ' + str(s_loc/len(user_loc)))


Количество локальных сессий для 89270693456
10
Количество локальных сессий для 89196230374
4
Количество локальных сессий для 89270698741
10
Количество локальных сессий для 89176732344
13
Количество локальных сессий для 89162735523
14
Количество локальных сессий для 89193729288
9
Количество локальных сессий для 89273028630
9
Число сессий: 69
Среднее число сессий на одного пользователя: 9.857142857142858


In [33]:
s_r = 0
data_1 = dataframe.loc[dataframe['type'] == 1]
user_r = data_1['user_id'].unique()
for i in user_r:
    print('Количество удалённых сессий для ' + str(i))
    print(list(data_1['session'].loc[(data_1['user_id'] == i)])[-1])
    s_r += int(list(data_1['session'].loc[(data_1['user_id'] == i)])[-1])
print('Число сессий: ' + str(s_r))
print('Среднее число сессий на одного пользователя: ' + str(s_r/len(user_r)))


Количество удалённых сессий для 80171109282
10
Количество удалённых сессий для 89273345194
18
Количество удалённых сессий для 89163828720
9
Число сессий: 37
Среднее число сессий на одного пользователя: 12.333333333333334
